In [ ]:
model.train()
loss.backward()
optimzer.step()

In [ ]:
import os
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https: // data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https: // data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https: // github.com/pyg-team/pytorch_geometric.git
!pip install torch-geometric-temporal

In [ ]:
!pip install pytorch-lightning  "pytorch-lightning[extra]" d2l == 0.17.0

In [1]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning|
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pytorch_lightning as pl

#d2l
import d2l.torch as d2l

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For Progress Bar
from tqdm import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

## Test

In [9]:
from torch_geometric_temporal.dataset import EnglandCovidDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = EnglandCovidDatasetLoader()
dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [15]:
from tqdm import tqdm
import torch
from torch_geometric_temporal.signal import temporal_signal_split
from model.dygrae import Dygre

loader = EnglandCovidDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

conv_out_channels = 8
conv_num_layers = 1
lstm_out_channels = 16
out_channels = 1

model = Dygre(conv_out_channels, conv_num_layers, lstm_out_channels, out_channels)

for param in model.parameters():
    param.retain_grad()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(100)):
    cost = 0
    H = torch.zeros((129, 16))   # TODO: hard code
    C = torch.zeros((129, 16))
    for time, snapshot in enumerate(train_dataset):
        y_hat, H, C = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, H, C)
        cost = cost + torch.sum((y_hat - snapshot.y) ** 2)

    cost = cost / (time + 1)

    if epoch % 20 == 0:
        print("MSE: {:.4f}".format(cost))

    cost.backward(retain_graph=True)
    optimizer.step()
    optimizer.zero_grad()

model.eval()
cost = 0
H = torch.zeros((129, 16))   # TODO: hard code
C = torch.zeros((129, 16))

for time, snapshot in enumerate(test_dataset):
    if time == 0:
        model.recurrent.weight = None
    y_hat, H, C = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, H, C)
    cost = cost + torch.mean((y_hat - snapshot.y) ** 2)
cost = cost / (time + 1)
cost = cost.item()
print("L1: {:.4f}".format(cost))

  2%|▏         | 2/100 [00:00<00:07, 12.27it/s]

MSE: 10641.8359


 25%|██▌       | 25/100 [00:01<00:02, 29.94it/s]

MSE: 8315.4121


 45%|████▌     | 45/100 [00:01<00:01, 32.36it/s]

...

MSE: 8036.9580


 87%|████████▋ | 87/100 [00:03<00:00, 30.21it/s]

MSE: 8055.3037


100%|██████████| 100/100 [00:03<00:00, 28.75it/s]


L1: 1.4779


In [14]:
from tqdm import tqdm
import torch
from torch_geometric_temporal.signal import temporal_signal_split
from model.gruconv import GRUConv

loader = EnglandCovidDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

model = GRUConv(8, 1)
for param in model.parameters():
    param.retain_grad()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.sum((y_hat - snapshot.y) ** 2)
    cost = cost / (time + 1)

    if epoch % 20 == 0:
        print("MSE: {:.4f}".format(cost))

    cost.backward(retain_graph=True)
    optimizer.step()
    optimizer.zero_grad()

model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    if time == 0:
        model.recurrent.weight = None
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat - snapshot.y) ** 2)
cost = cost / (time + 1)
cost = cost.item()
print("L1: {:.4f}".format(cost))

  1%|          | 2/200 [00:00<00:25,  7.76it/s]

MSE: 14588.7676


 11%|█         | 22/200 [00:02<00:18,  9.81it/s]

MSE: 10674.3301


 20%|██        | 41/200 [00:04<00:16,  9.68it/s]

...

MSE: 9398.9922


 91%|█████████ | 182/200 [00:14<00:01, 14.09it/s]

MSE: 9348.2998


100%|██████████| 200/200 [00:15<00:00, 12.78it/s]


L1: 0.8913


## Test(category)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MLP(hidden_channels=16).to(device)  # TODO: init model
criterion =  # TODO: Define loss criterion.
optimizer = torch.optim.Adam(model.parameters())  # Define optimizer.


def train(x, y, edge_index, mask):
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(x, edge_index)  # TODO: add edge index or weight
    loss = criterion(out[mask], y[mask])  # TODO: sure tensor shape and mask in loss
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss


def test(x, y, edge_index, mask):
    model.eval()
    out = model(x, edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    test_correct = pred[mask] == y[mask]  # TODO: Check against ground-truth labels.
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
    return test_acc


for epoch in range(1, 201):
    loss = train(data.x.to(device), data.y.to(device), data.edge_index.to(device))
    val_acc = test(data.x.to(device), data.y.to(device), data.edge_index.to(device), data.val_mask)
    test_acc = test(data.x.to(device), data.y.to(device), data.edge_index.to(device), data.test_mask)
    print(f'Epoch: {epoch:03d}')
    print(f'Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')